In [1]:
from tkinter import *
from tkinter.messagebox import showinfo
import numpy as np
import copy


In [2]:
#Coordinates go from 0 to 8
#Numbers go from 1 to 9, 0 means empty cell
#"pm": Pencilmark

#Auxiliary array for handling digits as bits
EXP2 = np.array([1,2,4,8,16,32,64,128,256,512,1024]).astype(np.int16)
BITNR = np.zeros((2048)).astype(np.int16)
for i in range(0,2048):
    for e2 in EXP2:
        if i&e2>0:
            BITNR[i]+=1

#Base of Grid class
#This contains only those information which is needed for recursive solution

class GridBase:   
    
    def __init__(self, orig = None):
        self.nr           = 0 #Number of digits in the grid
        self.grid         = np.zeros( (9,9) ).astype(np.int8)    #The grid itself                       
        self.options      = np.zeros( (9,9,10) ).astype(np.int8) #boolean for each option (i.e. centerpm)
        self.options_nr   = np.zeros( (9,9) ).astype(np.int8)    #nr of options        
        self.nr_digit_row = np.zeros( (9,10) ).astype(np.int8) #[row,digit]->number of digit in the row
        self.nr_digit_col = np.zeros( (9,10) ).astype(np.int8) #[column,digit]->number of digit in the column
        self.nr_digit_box = np.zeros( (9,10) ).astype(np.int8) #[box,digit]->number of digit in the box
        self.hints        = np.full((9,9), '', dtype=object) #Hints (naked single, lonely option)
        self.less         = [] #List of ordered cell-pairs (i0,j0,i1,j1), where cell 0 is smaller than cell 1

        if orig != None:
            self.nr = orig.nr
            self.less = copy.deepcopy(orig.less)
            for i in range(9):
                for d in range(1,10):
                    self.nr_digit_row[i,d] = orig.nr_digit_row[i,d]
                    self.nr_digit_col[i,d] = orig.nr_digit_row[i,d]
                    self.nr_digit_box[i,d] = orig.nr_digit_row[i,d]                
                for j in range(9):
                    self.grid[i,j]         = orig.grid[i,j]
                    self.options_nr[i,j]   = orig.options_nr[i,j]
                    for d in range(1,10):
                        self.options[i,j,d] = orig.options[i,j,d]
        else:        
            #Initializing member
            for i in range(9):
                for j in range(9):
                    self.grid[i,j]       = 0    #The cell is empty at the beginning
                    self.options_nr[i,j] = 9    #All options are available at the beginning -> #(1..9)=9
                    self.options[i,j,0]  = 0    #0 is never an option
                    d=j+1
                    self.nr_digit_row[i,d]    = 9 #There are 9 of each digit in each row   
                    self.nr_digit_col[i,d]    = 9 #There are 9 of each digit in each column
                    self.nr_digit_box[i,d]    = 9 #There are 9 of each digit in each box
                    for k in range(1,10):
                        self.options[i,j,k] = 1 #All options are available at the beginning
                
                    
    def get_digit(self,i,j):
        return self.grid[i,j]
    
    
    def get_options(self,i,j):
        res = []
        for k in range(1,10):
            if self.options[i,j,k]==1:
                res.append(k)
        return res

    def get_less(self):
        return self.less
    
    def is_option(self,i,j,digit):
        return self.options[i,j,digit]
    
    def add_digit(self, i, j, digit):
        if self.options[i,j,digit]==0:
            return False
        self.nr += 1
        self.grid[i,j]=digit
        self.update_options()
        self.update_hints()
        return True        
    
    def delete_option(self,i,j,digit,update_hints=True, update_less=True):
        if self.options[i,j,digit]==1:
            self.options[i,j,digit] = 0
            self.options_nr[i,j] -= 1
            self.update_nr_digits(i,j,digit)
            if update_hints:
                self.update_hints()
            if update_less:
                self.update_less()
            return True
        else:
            return False
    
    def update_options(self):
        for i in range(9):
            for j in range(9):
                if self.grid[i,j]!=0:                    
                    d = self.grid[i,j]
                    for dd in range(1,10):
                        if self.options[i,j,dd]==1:
                            self.options[i,j,dd] = 0                        
                            self.options_nr[i,j]-= 1   
                            self.update_nr_digits(i,j,dd)                            
                    for k in range(9):
                        if k!=j:
                            if self.options[i,k,d] == 1:
                                self.options[i,k,d] = 0
                                self.options_nr[i,k] -= 1  
                                self.update_nr_digits(i,k,d)
                        if k!=i:
                            if self.options[k,j,d] == 1:
                                self.options[k,j,d] = 0
                                self.options_nr[k,j] -= 1
                                self.update_nr_digits(k,j,d)
                    i0 = i - i%3
                    j0 = j - j%3
                    for ii in range(3):
                        for jj in range(3):
                            if i0+ii!=i or j0+jj!=j:
                                if self.options[i0+ii,j0+jj,d]==1:
                                    self.options[i0+ii,j0+jj,d]=0
                                    self.options_nr[i0+ii,j0+jj]-=1
                                    self.update_nr_digits(i0+ii,j0+jj,d)
        self.update_less()
                                   
    def update_less(self):
        if self.less:
            changed = True
            while changed:
                changed = False
                for (i0,j0,i1,j1) in self.less:
                    min0,max1=9,1
                    if self.grid[i0,j0]==0:
                        for d in range(1,10):
                            if self.options[i0,j0,d]:
                                min0 = d
                                break
                    else:
                        min0 = self.grid[i0,j0]
                    if self.grid[i1,j1]==0:
                        for d in range(9,0,-1):
                            if self.options[i1,j1,d]:
                                max1 = d
                                break
                    else:
                        max1 = self.grid[i1,j1]                    
                    for d in range(1,min0+1):
                        if self.options[i1,j1,d]:
                            self.delete_option(i1,j1,d,update_hints=False,update_less=False)
                            changed = True
                    for d in range(max1,10):
                        if self.options[i0,j0,d]:
                            self.delete_option(i0,j0,d,update_hints=False,update_less=False)
                            changed = True                                        
            self.update_hints()
                                
    def update_nr_digits(self, i, j, digit):
        self.nr_digit_row[i,digit]-=1
        self.nr_digit_col[j,digit]-=1
        self.nr_digit_box[(i//3)*3+j//3,digit]-=1                                    

    #Here we handle only the one-digit deductions which directly result a new digit into the grid
    def update_hints(self):
        hnr = 0
        self.hints = np.full((9,9), '', dtype=object)        
        #Naked singles
        for i in range(9):
            for j in range(9):
                if self.grid[i,j]==0 and self.options_nr[i,j]==1:
                    self.hints[i,j]+='NS;'
                    hnr += 1
                    
        #Only one place in row
        for i in range(9):
            for d in range(1,10):
                if self.nr_digit_row[i,d]==1:
                    for j in range(9):
                        if self.options[i,j,d]==1 and self.hints[i,j]=='':
                            self.hints[i,j]+='C'+str(d)+';'
                            hnr += 1
                            break
        #Only one place in col
        for j in range(9):
            for d in range(1,10):
                if self.nr_digit_col[j,d]==1:
                    for i in range(9):
                        if self.options[i,j,d]==1 and self.hints[i,j]=='':
                            self.hints[i,j]+='R'+str(d)+';'
                            hnr += 1
                            break
        #Only one place in box
        for k in range(9):
            for d in range(1,10):
                if self.nr_digit_box[k,d]==1:
                    for ii in range(3):
                        for jj in range(3):
                            i=(k//3)*3+ii
                            j=(k%3)*3+jj
                            if self.options[i,j,d]==1 and self.hints[i,j]=='':
                                self.hints[i,j]+='B'+str(d)+';'
                                hnr += 1
                                break
        return hnr

    def is_broken(self):
        res = False
        for i in range(9):
            for j in range(9):
                if self.grid[i,j]==0 and self.options_nr[i,j]==0:
                    res = True
        return res

    def is_solved(self):
        res = True
        for i in range(9):
            for j in range(9):
                if self.grid[i,j]==0:
                    res = False
        return res 


In [3]:
MAXSOLNR  = 8
MAXNODENR = 100
class Solver:
    def __init__(self, grid):
        self.nr     = 0
        self.first  = None
        self.grid   = grid
        self.nodenr = 0
    
    # Returning values:
    # * Number of solutions. If it is MAXSOLNR -> there can be more solutions 
    # * If the number of solutions is exact, provided it is less than MAXSOLNR
    # * Grid of the first solution found
    def solve(self):
        self.nr = 0
        self.first  = None
        self.nodenr = 0
        g = GridBase(self.grid)
        self.rec(g)
        return self.nr, self.nodenr < MAXNODENR, self.first

    def update(self,g):
        hnr = g.update_hints()
        while (hnr!=0):
            for i in range(9):
                for j in range(9):
                    if len(g.hints[i,j])>=3:
                        if g.hints[i,j][0:3]=='NS;':
                            for d in range(1,10):
                                if g.options[i,j,d]==1:
                                    g.add_digit(i,j,d)
                                    break
                        else: #Only one in row/column/box (in that case the 2nd character contains the digit)
                            g.add_digit(i,j, int(g.hints[i,j][1]) )
            hnr = g.update_hints()                
        
    def rec(self, g):
        if self.nr >= MAXSOLNR:
            return
        self.nodenr += 1
        if self.nodenr >= MAXNODENR:
            return
        self.update(g)
        opnr=10
        ii,jj=0,0
        for i in range(9):
            for j in range(9):
                if g.options_nr[i,j]>0 and g.options_nr[i,j]<opnr:
                    ii,jj,opnr = i,j,g.options_nr[i,j]
        if opnr==10: #No further options -> Either we have a solution, or we have a dead end
            if g.is_solved():
                self.nr += 1
                if self.first == None:
                    self.first = copy.deepcopy(g)                    
            return 
        for d in range(1,10):
            if g.options[ii,jj,d]==1:
                g2 = copy.deepcopy(g)
                g2.add_digit(ii,jj,d)
                self.rec(g2)
        
        
        

In [4]:
#Number of supported 1-cell shapes (including no-shape)
#0 Nothing
#1 Filled circle
#2 Empty circle
#3 Filled square
#4 Empty square
#5 Min cell
#6 Max cell   
#7 Encircled cell (for summing arrows)
SHAPE1NR = 8 

#Number of supported 2-cell shapes (including no-shape)
#0 Nothing
#1 Black Kropki dot
#2 White Kropki dot
#3 Black Kropki square
#4 White Kropki square
#5 X
#6 V
#7 <
#8 >
#9 Encircled double cell (for summing arrows)
SHAPE2NR = 10

#Number of line segment types (including no-shape)
#0 Nothing
#1 Thermo line
#2 Thin line
#3 Thin line with arrow at the beginning
#4 Thin line with arrow at the end
LINESNR = 5

class Grid(GridBase):   
    
    def __init__(self, orig = None):
        
        super().__init__()
        
        #boolean for indicating digits belonging to the puzzle (and not the solution)
        self.orig   = np.zeros( (9,9) ).astype(np.int8) 

        #Additional options-related information
        self.options_ch = np.zeros( (9,9) ).astype(np.int8)    #boolean if options were changed (by hand) in a cell 
        self.options_sh = np.zeros( (9,9) ).astype(np.int8)    #boolean if options are shown in a cell 

        # Top Pencilmark
        self.toppm      = np.zeros( (9,9,10) ).astype(np.int8) #boolean for each option
        
        #Coloring (during solution)
        self.color      = np.zeros( (9,9) ).astype(np.int8)    #coloring of the cells; 0 -> no coloring
        
        #Showing options automatically if their number <= self.showlevel
        self.showlevel = 3
        
        #1-cell shapes: List of (type, i, j);
        self.shape1 = []

        #2-cell shapes: List of (type, i1, j1, i2, j2); 
        self.shape2 = []
        
        #Line segments behave as 2-cell shapes: (type, i1, j1, i2, j2)
        self.lines = []
               
        #Initializing members
        for i in range(9):
            for j in range(9):
                self.orig[i,j]       = 0    #The grid is empty at the beginning (no given digits)
                self.options_sh[i,j] = 0    #Not showing the options at the beginning
                self.options_ch[i,j] = 0    #No change at the beginning 
                self.toppm[i,j,0]    = 0    #0 is never a top pm
                self.color[i,j]      = 0    #No coloring for each cell
                for k in range(1,10):
                    self.toppm[i,j,k]   = 0 #There is no top pm at the beginning                                   
    
    def copydigits(self,other):
        for i in range(9):
            for d in range(1,10):
                self.nr_digit_row[i,d] = 0
                self.nr_digit_col[i,d] = 0
                self.nr_digit_box[i,d] = 0
            for j in range(9):
                self.grid[i,j] = other.grid[i,j]
                self.options_ch[i,j] = 1
                self.options_sh[i,j] = 1
                self.options_nr[i,j] = 0
                self.hints[i,j]      = ''
                for d in range(1,10):
                    self.options[i,j,d] = 0
                    self.toppm[i,j,d]   = 0
    
    def is_orig(self,i,j):
        return self.orig[i,j]

    def get_centerpm(self,i,j,forced=False):
        res = ''
        if self.options_sh[i,j] or forced:
            for k in range(1,10):
                if self.options[i,j,k]==1:
                    res += str(k)
        return res

    def is_centerpm(self,i,j,digit):
        return self.options[i,j,digit]

    def get_toppm(self,i,j):
        res = ''
        for k in range(1,10):
            if self.toppm[i,j,k]==1:
                res += str(k)
        return res

    def is_toppm(self,i,j,digit):
        return self.toppm[i,j,digit]
    
    def get_color(self, i, j):
        return self.color[i,j]

    def get_shape1(self):
        return self.shape1

    def get_shape2(self):
        return self.shape2

    def get_lines(self):
        return self.lines
    
    def add_digit(self, i, j, digit, orig=False):
        res = super().add_digit(i,j,digit)
        if res:
            if orig:
                self.orig[i,j]=1
            self.update_show()            
        return res
    
    def delete_centerpm(self,i,j,digit):
        #If not visible -> no change
        if not self.options_sh[i,j]:
            return False
        res = super().delete_option(i,j,digit)
        if res:
            self.options_ch[i,j] = 1
            self.toppm[i,j,digit]=0
            self.update_show()
        return res
            
    def is_centerpm_shown(self,i,j):
        return self.options_sh[i,j]
            
    def toggle_centerpm_shown(self,i,j):        
        if self.options_sh[i,j] and self.options_ch[i,j]:
            return False
        self.options_sh[i,j] = 1 - self.options_sh[i,j]           
        return True
            
    def change_toppm(self,i,j,digit):
        if self.toppm[i,j,digit]==1:
            self.toppm[i,j,digit] = 0
            return True
        else:
            valid=self.options[i,j,digit]
            if valid:
                self.toppm[i,j,digit]=1
            return valid
    
    def change_color(self,i,j,color):
        res = False
        if self.color[i,j] != color:
            self.color[i,j] = color
            res = True
        return res
    
    def update_options(self):
        super().update_options()        
        for i in range(9):
            for j in range(9):
                if self.grid[i,j]!=0:                    
                    d = self.grid[i,j]
                    for dd in range(1,10):
                        self.toppm[i,j,dd]=0                            
                    for k in range(9):
                        if k!=j:
                            self.toppm[i,k,d]=0
                        if k!=i:
                            self.toppm[k,j,d]=0
                    i0 = i - i%3
                    j0 = j - j%3
                    for ii in range(3):
                        for jj in range(3):
                            self.toppm[i0+ii,j0+jj,d]=0
        self.update_show()
                                    
    def update_hints(self):
        #Super handles Naked singles & Only one place in row/column/box
        hnr = super().update_hints()
        
        #pair, tripple, quadruple, quintuple
        for nr in range(2,min(self.showlevel,5)+1):
            for i in range(9):
                binary_col = np.zeros((9)).astype(np.int16)
                for j in range(9):
                    for d in range(1,10):
                        binary_col[j] |= self.options[i,j,d]*EXP2[d]
                hnr += self.tuple_col(i,binary_col,nr,0,0,0,[])
            for j in range(9):
                binary_row = np.zeros((9)).astype(np.int16)
                for i in range(9):
                    for d in range(1,10):
                        binary_row[i] |= self.options[i,j,d]*EXP2[d]
                hnr += self.tuple_row(j,binary_row,nr,0,0,0,[])
            for b in range(9):
                binary_box = np.zeros((9)).astype(np.int16)
                for i in range(9):
                    ii = (b//3)*3 + i//3
                    jj = (b%3)*3 + i%3
                    for d in range(1,10):
                        binary_box[i] |= self.options[ii,jj,d]*EXP2[d]
                hnr += self.tuple_box(b,binary_box,nr,0,0,0,[])
                
        return hnr
            
                         
    #ci: index of current column
    #l: list containing the column in binarized form
    #nr: size of tuple; 
    #k: k digits so far; 
    #p: first option for k+1-th digit
    #disj: disjunction
    #places: those places which contain the tuple elements
    def tuple_col(self,ci, l, nr, k, p, disj, places):
        hnr = 0
        if k==nr:
            #Check if the tuple found is valuable (it exists in other cell)
            for i in range(9):
                if i not in places:
                    if l[i]&disj>0:
                        for p in places:
                            if 'TC' not in self.hints[ci,p]:
                                self.hints[ci,p]+='TC'+str(nr)+';'
                                hnr += 1
                        break
            return
        for i in range(p,9-nr+k+1):
            if BITNR[l[i]]<=1 or BITNR[l[i]]>nr:
                continue
            newdisj = disj | l[i]
            if BITNR[newdisj]>nr:
                continue
            self.tuple_col(ci,l,nr,k+1,i+1,newdisj,places+[i])            
        return hnr
            
    #ri: index of current row
    #l: list containing the row in binarized form
    #nr: size of tuple; 
    #k: k digits so far; 
    #p: first option for k+1-th digit
    #disj: disjunction
    #places: those places which contain the tuple elements
    def tuple_row(self, ri, l, nr, k, p, disj, places):
        hnr = 0
        if k==nr:
            #Check if the tuple found is valuable (it exists in other cell)
            for i in range(9):
                if i not in places:
                    if l[i]&disj>0:
                        for p in places:
                            if 'TR' not in self.hints[p,ri]:
                                self.hints[p,ri]+='TR'+str(nr)+';'
                                hnr += 1
                        break
            return    
        for i in range(p,9-nr+k+1):
            if BITNR[l[i]]<=1 or BITNR[l[i]]>nr:
                continue
            newdisj = disj | l[i]
            if BITNR[newdisj]>nr:
                continue
            self.tuple_row(ri,l,nr,k+1,i+1,newdisj,places+[i])            
        return hnr

    #bi: index of current box
    #l: list containing the box in binarized form
    #nr: size of tuple; 
    #k: k digits so far; 
    #p: first option for k+1-th digit
    #disj: disjunction
    #places: those places which contain the tuple elements
    def tuple_box(self, bi, l, nr, k, p, disj, places):
        hnr = 0
        i0=(bi//3)*3
        j0=(bi%3)*3
        if k==nr:
            #Check if the tuple found is valuable (it exists in other cell)
            for i in range(9):
                if i not in places:
                    if l[i]&disj>0:
                        for p in places:
                            if 'TB' not in self.hints[i0+p//3,j0+p%3]:
                                self.hints[i0+p//3,j0+p%3]+='TB'+str(nr)+';'
                                hnr += 1
                        break
            return    
        for i in range(p,9-nr+k+1):
            if BITNR[l[i]]<=1 or BITNR[l[i]]>nr:
                continue
            newdisj = disj | l[i]
            if BITNR[newdisj]>nr:
                continue
            self.tuple_box(bi,l,nr,k+1,i+1,newdisj,places+[i])            
        return hnr
            
            
    def update_show(self):
        for i in range(9):
            for j in range(9):
                if self.options_nr[i,j] <= self.showlevel:
                    self.options_sh[i,j] = 1
   
    def change_showlevel(self, delta):
        res = False
        if 0<=self.showlevel+delta<=9:
            self.showlevel += delta
            for i in range(9):
                for j in range(9):
                    if self.options_sh[i,j] and \
                       not self.options_ch[i,j] and \
                       self.options_nr[i,j] > self.showlevel:
                        self.options_sh[i,j]=0
                    elif not self.options_sh[i,j] and \
                         self.options_nr[i,j] <= self.showlevel:
                        self.options_sh[i,j]=1
            res = True
        return res

    def next_1cellshape_digit(self,i,j):
        ind = -1
        for k,s in enumerate(self.shape1):
            if s[1]==i and s[2]==j:
                ind = k
                break
        if ind==-1:
            self.shape1.append( [1,i,j] )
        else:
            self.shape1[ind][0] = (self.shape1[ind][0] + 1) % SHAPE1NR
        return True

    def next_2cellshape_digit(self,i1,j1,i2,j2):
        #Cells are ordered, only orthogonally adjacent cells are allowed
        if i1>i2 or j1>j2 or (j2-j1+i2-i1)!=1: 
            return False
        ind = -1        
        for k,s in enumerate(self.shape2):
            if s[1]==i1 and s[2]==j1 and s[3]==i2 and s[4]==j2:
                ind = k
                break
        if ind==-1:
            self.shape2.append( [1,i1,j1,i2,j2] )
        else:
            self.shape2[ind][0] = (self.shape2[ind][0] + 1) % SHAPE2NR
        return True

    def next_line_segment(self,i1,j1,i2,j2):
        #Cells are ordered, only 8-way adjacent cells are allowed
        if i1>i2 or (i1==i2 and j1>j2) or (i2-i1)>1 or abs(j2-j1)>1:
            return False
        ind = -1        
        for k,s in enumerate(self.lines):
            if s[1]==i1 and s[2]==j1 and s[3]==i2 and s[4]==j2:
                ind = k
                break
        if ind==-1:
            self.lines.append( [1,i1,j1,i2,j2] )
        else:
            self.lines[ind][0] = (self.lines[ind][0] + 1) % LINESNR
        return True
    
    def build_thermo(self):        
        bulbs = set()
        for s1 in self.shape1:
            if s1[0]==1:
                bulbs.add( (s1[1],s1[2]) )
        thermolines = set()
        for s2 in self.lines:
            if s2[0]==1:
                thermolines.add( ((s2[1],s2[2]),(s2[3],s2[4])) )               
        lesscells = set()
        for l in self.less:
            lesscells.add( (l[0],l[1]) )
            lesscells.add( (l[2],l[3]) )
            if ((l[0],l[1]),(l[2],l[3])) in thermolines:
                thermolines.remove( ((l[0],l[1]),(l[2],l[3])) )
            if ((l[2],l[3]),(l[0],l[1])) in thermolines:
                thermolines.remove( ((l[2],l[3]),(l[0],l[1])) )
        changed = True
        while changed:
            changed = False
            for t in thermolines:
                if t[0] in lesscells and t[1] not in lesscells and t[1] not in bulbs:
                    self.less.append( (t[0][0],t[0][1],t[1][0],t[1][1]) )
                    lesscells.add(t[1])                    
                    changed=True
                    break
                elif t[0] not in lesscells and t[1] in lesscells and t[0] not in bulbs:
                    self.less.append( (t[1][0],t[1][1],t[0][0],t[0][1]) )
                    lesscells.add(t[0])                    
                    changed=True
                    break
                elif t[0] not in lesscells and t[1] not in lesscells:
                    if t[0] in bulbs and t[1] not in bulbs:
                        self.less.append( (t[0][0],t[0][1],t[1][0],t[1][1]) )
                        lesscells.add(t[0])
                        lesscells.add(t[1])                        
                        changed=True
                        break
                    elif t[1] in bulbs and t[0] not in bulbs:
                        self.less.append( (t[1][0],t[1][1],t[0][0],t[0][1]) )
                        lesscells.add(t[0])
                        lesscells.add(t[1])                        
                        changed=True
                        break
            if changed:
                thermolines.remove(t)
        self.update_options()
        self.update_hints()                
        return True        
        
                

In [5]:
class GridList:
    def __init__(self):    
        grid = Grid()
        
        #List of grids, always pointing to the current list in self.listlist
        #This is the undo-redo list for solving the sudoku from one particular starting position
        self.gridlist = [grid]

        #List of girdlists for the different starting positions (including the current pointer [curri])
        #This is the undo-redo list for the different starting positions 
        #including the gridlists belonging to them
        self.listlist = [ [self.gridlist,0] ]
        
        self.listi = 0 #Index of current list
        self.curri = 0 #Index of current grid in the current list

    def appendgrid(self,newgrid):
        del self.gridlist[self.curri+1:]
        self.gridlist.append(newgrid)
        self.curri = len(self.gridlist)-1
        
        
    def getcurr(self):
        res =  self.gridlist[self.curri]
        return res 
        
    def add_digit(self, selected, i, j, digit):
        currgrid = self.gridlist[self.curri]
        newgrid  = copy.deepcopy(currgrid) 
        success = True
        if selected:
            for (ii,jj) in selected:                
                if not newgrid.add_digit(ii,jj,digit):
                    success = False
                    break
        else:
            success = newgrid.add_digit(i,j,digit)
        if success:
            self.appendgrid(newgrid)

    def delete_centerpm(self, selected, i, j, digit):
        currgrid = self.gridlist[self.curri]
        newgrid  = copy.deepcopy(currgrid) 
        success = False
        if selected: 
            nr = 0
            for (ii,jj) in selected:
                if newgrid.is_option(ii,jj,digit):
                    nr += 1
            for (ii,jj) in selected:
                if not newgrid.is_centerpm_shown(ii,jj):
                    success = False # No change if any CenterPM is invisible in the selection
                    break
                    #newgrid.toggle_centerpm_shown(ii,jj)
                if nr>0:
                    if newgrid.is_option(ii,jj,digit):
                        success |= newgrid.delete_centerpm(ii,jj,digit)                                    
                else:
                    if not newgrid.is_option(ii,jj,digit):
                        success |= newgrid.delete_centerpm(ii,jj,digit)                    
        else: #no selection: using currently framed cell
            success = newgrid.delete_centerpm(i,j,digit)
        if success:
            self.appendgrid(newgrid)

    def change_toppm(self, selected, i, j, digit):
        currgrid = self.gridlist[self.curri]
        newgrid  = copy.deepcopy(currgrid) 
        success = False
        if selected: 
            nr=0
            for (ii,jj) in selected:
                if newgrid.is_toppm(ii,jj,digit):
                    nr += 1
            for (ii,jj) in selected:
                if nr>0:
                    if newgrid.is_toppm(ii,jj,digit):
                        success |= newgrid.change_toppm(ii,jj,digit)                    
                else:
                    if not newgrid.is_toppm(ii,jj,digit):
                        success |= newgrid.change_toppm(ii,jj,digit)                    
        else:
            success = newgrid.change_toppm(i,j,digit)
        if success:
            self.appendgrid(newgrid)
            
    def change_color(self, selected, i, j, color):
        currgrid = self.gridlist[self.curri]
        newgrid  = copy.deepcopy(currgrid) 
        success = False
        if selected:             
            for (ii,jj) in selected:
                success |= newgrid.change_color(ii,jj,color)
        else:
            success = newgrid.change_color(i,j,color)
        if success:
            self.appendgrid(newgrid)        
           
    def centerpm_onoff(self, selected, i, j):
        currgrid = self.gridlist[self.curri]
        newgrid  = copy.deepcopy(currgrid) 
        success = False
        if selected: 
            #Success: If at least one cell was changed
            #If not all shown -> 
            #  1: show all
            #  2: then delete those which can be deleted
            nr=0
            for (ii,jj) in selected:
                if not newgrid.is_centerpm_shown(ii,jj):
                    nr += 1
            for (ii,jj) in selected:
                if nr>0:
                    if not newgrid.is_centerpm_shown(ii,jj):
                        success |= newgrid.toggle_centerpm_shown(ii,jj)
                else:
                    if newgrid.is_centerpm_shown(ii,jj):
                        success |= newgrid.toggle_centerpm_shown(ii,jj)
        else:
            success = newgrid.toggle_centerpm_shown(i,j)
        if success:
            self.appendgrid(newgrid)
            
    def undo(self):
        if self.curri>0:
            self.curri -= 1
    
    def redo(self):
        if self.curri<len(self.gridlist)-1:
            self.curri += 1
    
    def is_solved(self):
        return self.gridlist[self.curri].is_solved()
        
    def is_broken(self):
        return self.gridlist[self.curri].is_broken()

    def change_showlevel(self,delta):
        currgrid = self.gridlist[self.curri]
        newgrid  = copy.deepcopy(currgrid) 
        success = newgrid.change_showlevel(delta)
        if success:
            self.appendgrid(newgrid)    
    
    def appendlist(self, newlist):
        del self.listlist[self.listi+1:]
        self.listlist[self.listi][1] = self.curri #Saving current self.curri
        self.listlist.append( [newlist,self.curri] )
        self.gridlist = newlist
        self.listi = len(self.listlist)-1                        
    
    #Adding a new digit to the starting position
    # -> This starts a new gridlist in listlist
    def add_base_digit(self, selected, i, j, digit):
        currlist = self.gridlist
        newlist  = copy.deepcopy(self.gridlist)
        del newlist[self.curri+1:]
        if selected:
            for (ii,jj) in selected:
                for grid in newlist:
                    success = grid.add_digit(ii,jj,digit,orig=True)
                    if not success:
                        break
                if not success:
                    break
        else:        
            success = True
            for grid in newlist:
                success = grid.add_digit(i,j,digit,orig=True)
                if not success:
                    break                
        if success:
            self.appendlist(newlist)
    
    def undo_base(self):
        if self.listi>0:
            self.listlist[self.listi][1] = self.curri #Saving curri
            self.listi -= 1
            self.gridlist, self.curri = self.listlist[self.listi]                    
    
    def redo_base(self):
        if self.listi<len(self.listlist)-1:
            self.listlist[self.listi][1] = self.curri #Saving curri
            self.listi += 1
            self.gridlist, self.curri = self.listlist[self.listi]
    
    def next_1cellshape_base(self, i, j):
        currlist = self.gridlist
        newlist  = copy.deepcopy(self.gridlist)
        del newlist[self.curri+1:]
        success = True
        for grid in newlist:
            success = grid.next_1cellshape_digit(i,j)
            if not success:
                break
        if success:            
            self.appendlist(newlist)
        
    def next_2cellshape_base(self, i1, j1, i2, j2):
        currlist = self.gridlist
        newlist  = copy.deepcopy(self.gridlist)
        del newlist[self.curri+1:]
        success = True
        for grid in newlist:
            success = grid.next_2cellshape_digit(i1,j1,i2,j2)
            if not success:
                break
        if success:
            self.appendlist(newlist)
            
    def next_line_segment_base(self, i1, j1, i2, j2):
        currlist = self.gridlist
        newlist  = copy.deepcopy(self.gridlist)
        del newlist[self.curri+1:]
        success = True
        for grid in newlist:
            success = grid.next_line_segment(i1,j1,i2,j2)
            if not success:
                break
        if success:
            self.appendlist(newlist)
            
    def find_solution(self):
        currgrid = self.gridlist[self.curri]
        solver = Solver(currgrid)
        nr,exact,grid = solver.solve()   
        if grid != None:
            newgrid  = copy.deepcopy(currgrid) 
            newgrid.copydigits(grid)
            self.appendgrid(newgrid)
        return nr, exact
    
    def count_solution(self):
        currgrid = self.gridlist[self.curri]
        solver = Solver(currgrid)
        nr,exact,_ = solver.solve()   
        return nr, exact
    
    def build_thermo_base(self):
        currlist = self.gridlist
        newlist  = copy.deepcopy(self.gridlist)
        del newlist[self.curri+1:]
        success = True
        for grid in newlist:
            success = grid.build_thermo()
            if not success:
                break
        if success:
            self.appendlist(newlist)
        
    

In [6]:
VP=HP=65
VW=HW=65
TOPPM_PADDING = HP//3
BOTTOMPM_PADDING = HP//3
PM_ADJ = 2
LW1 = 2
LW2 = 3
FRAMEWIDTH=3
SHAPEWIDTH=8
SARROW=15

FONT_L        = "Mono"
FONTSIZE_L    = "9"
FONT          = "Mono"
FONTSIZE      = "40"
FONT_PM       = "Mono"
FONTSIZE_PM   = "10"
FONT_S2       = "Mono"
FONTSIZE_S2   = "12"
FONT_HINT     = "Mono"
FONTSIZE_HINT = "6"

SOLVEDCOLOR      = 'green'
BROKENCOLOR      = 'red'
BGCOLOR          = 'white'
LINECOLOR        = 'black'
FONTCOLOR        = 'darkblue'
ORIGFONTCOLOR    = 'black'
FRAMECOLOR       = 'red'
SELCOLOR         = 'yellow'
SELSTIPPLE       = 'gray50'
SHAPECOLOR       = 'gray50'
HINTCOLOR        = 'darkgreen'
INACTIVECPMCOLOR = 'lightgray'
WRONGTHERMO      = 'rosybrown1'

COLORLIST  = [BGCOLOR,'tomato','pale green',
              'deep sky blue','orange','medium orchid',
              'bisque4','cyan3','light goldenrod yellow','hot pink']
COLSTIPPLE = '' #'gray75'
COLW       = 15

class UI:
    
    def __init__(self, canvas):
        self.canvas = canvas
        self.framelines = []
        
    def canvas_pack(self):
        self.canvas.pack()
        
    
    def DrawBase(self, gridlist, currcolor):
        color = LINECOLOR
        if gridlist.is_solved():
            color = SOLVEDCOLOR
        elif gridlist.is_broken():
            color = BROKENCOLOR
        for i in range(10):
            w=LW1
            if i%3==0:
                w=LW2
            self.canvas.create_line(VP, HP+HW*i, VP+9*VW, HP+HW*i, fill=color, width=w)
        for i in range(10):
            w=LW1
            if i%3==0:
                w=LW2
            self.canvas.create_line(VP+VW*i, HP, VP+VW*i, HP+HW*9, fill=color, width=w)

        # Current color
        self.canvas.create_rectangle(2, 2, COLW, COLW, outline=LINECOLOR, width=1,
                                fill=COLORLIST[currcolor] )        
        
        #GridList info-s
        txt = "{}/{};{}/{};sh={};".format(gridlist.curri+1,len(gridlist.gridlist),
                                          gridlist.listi+1,len(gridlist.listlist),
                                          gridlist.gridlist[gridlist.curri].showlevel)
        self.canvas.create_text(1+VW//3+1,1,anchor = "nw",
                           fill=FONTCOLOR,
                           font=FONT_L+" "+FONTSIZE_L,
                           text=txt)        

    def DrawColors(self, grid):
        for i in range(9):
            for j in range(9):                
                t=VP+VW*i
                l=HP+HW*j
                b=VP+VW*(i+1)
                r=HP+HW*(j+1)
                colorind = grid.get_color(i,j)
                color = COLORLIST[ colorind ]
                self.canvas.create_rectangle(t, l, b, r, fill=color, outline='', stipple=COLSTIPPLE)        

                
    def DrawShapes(self, grid):        
        #Line segments: List of (type, i1, j1, i2, j2);
        for s in grid.get_lines():
            t0=VP+VW*s[1]+(VW+LW1)//2
            l0=HP+HW*s[2]+(HW+LW1)//2
            t1=VP+VW*s[3]+(VW+LW1)//2
            l1=HP+HW*s[4]+(HW+LW1)//2            
            if s[0]==0:   #0 No 2-cell shape -> nothing to draw
                pass
            elif s[0]==1:   #0 Thermo
                color = SHAPECOLOR
                less = grid.get_less()
                if (s[1],s[2],s[3],s[4]) not in less and (s[3],s[4],s[1],s[2]) not in less:
                    color = WRONGTHERMO
                sw2 = SHAPEWIDTH//2-1
                if (s[1]!=s[3] and s[2]!=s[4]): #diagonal
                    sw2 = int(sw2/1.41)
                self.canvas.create_rectangle(t0-sw2-1,l0-sw2-1,t0+sw2,l0+sw2, fill=color, outline=color)
                self.canvas.create_rectangle(t1-sw2-1,l1-sw2-1,t1+sw2,l1+sw2, fill=color, outline=color)
                self.canvas.create_line(t0, l0, t1, l1, fill=color, width=SHAPEWIDTH)                
            elif s[0]==2:   #0 Thin line
                self.canvas.create_line(t0, l0, t1, l1, fill=SHAPECOLOR, width=LW1)                
            elif s[0]==3:   #0 Thin line with arrow at the beginning
                self.canvas.create_line(t0, l0, t1, l1, fill=SHAPECOLOR, width=LW1,arrow='first')                
            elif s[0]==4:   #0 Thin line with arrow at the end
                self.canvas.create_line(t0, l0, t1, l1, fill=SHAPECOLOR, width=LW1,arrow='last')                                                
        #2-cell shapes: List of (type, i1, j1, i2, j2); 
        for s in grid.get_shape2():
            t0=VP+VW*s[1]
            l0=HP+HW*s[2]
            if s[1]!=s[3]:
                t0 += VW
            else:
                t0 += VW//2
            if s[2]!=s[4]:
                l0 += HW
            else:
                l0 += HW//2
            t=t0-VW//8
            l=l0-HW//8
            b=t0+VW//8
            r=l0+HW//8
            
            if s[0]==0:     #No 2-cell shape -> nothing to draw
                pass
            elif s[0]==1:   #Black Kropki dot
                self.canvas.create_oval(t,l,b,r, fill=LINECOLOR, outline=LINECOLOR, width=LW1)
            elif s[0]==2:   #White Kropki dot
                self.canvas.create_oval(t,l,b,r, fill=BGCOLOR, outline=LINECOLOR, width=LW1)
            elif s[0]==3:   #Black Kropki Square
                self.canvas.create_rectangle(t,l,b,r, fill=LINECOLOR, outline=LINECOLOR, width=LW1)
            elif s[0]==4:   #White Kropki Square
                self.canvas.create_rectangle(t,l,b,r, fill=BGCOLOR, outline=LINECOLOR, width=LW1)
            elif s[0]==5:   #X
                self.canvas.create_oval(t,l,b,r, fill=BGCOLOR, outline='')
                self.canvas.create_text(t0,l0,
                   anchor = "center",
                   fill=FONTCOLOR,
                   font=FONT_S2+" "+FONTSIZE_S2+" bold",
                   text='X')
            elif s[0]==6:   #V
                self.canvas.create_oval(t,l,b,r, fill=BGCOLOR, outline='')
                self.canvas.create_text(t0,l0,
                   anchor = "center",
                   fill=FONTCOLOR,
                   font=FONT_S2+" "+FONTSIZE_S2+" bold",
                   text='V')
            elif s[0]==7:   #<
                if s[2]==s[4]:
                    s = '<'
                else:
                    s = '∨'                
                self.canvas.create_text(t0,l0,
                   anchor = "center",
                   fill=FONTCOLOR,
                   font=FONT_S2+" "+FONTSIZE_S2+" bold",
                   text=s)
            elif s[0]==8:   #>
                if s[2]==s[4]:
                    s = '>'
                else:
                    s = '∧'
                self.canvas.create_text(t0,l0,
                   anchor = "center",
                   fill=FONTCOLOR,
                   font=FONT_S2+" "+FONTSIZE_S2+" bold",
                   text=s)
            elif s[0]==9:   #Encircled double-cell for summing arrows
                t0=VP+VW*s[1]
                l0=HP+HW*s[2]
                t1=VP+VW*(s[3]+1)
                l1=HP+HW*(s[4]+1)                
                self.canvas.create_polygon([t0+2*LW2,l0+2*LW2,
                                       (t0+t1)//2,l0+2*LW2,
                                       t1-2*LW2,l0+2*LW2,
                                       t1-2*LW2,(l0+l1)//2,
                                       t1-2*LW2,l1-2*LW2,
                                       (t0+t1)//2,l1-2*LW2,
                                       t0+2*LW2,l1-2*LW2,
                                       t0+2*LW2,(l0+l1)//2], 
                                      outline=SHAPECOLOR, fill=BGCOLOR, width=LW1, 
                                      smooth=True)
            
        # 1-cell Shapes
        for s in grid.get_shape1():
            t0=VP+VW*s[1]
            l0=HP+HW*s[2]
            t=VP+VW*s[1]+VW//4
            l=HP+HW*s[2]+HW//4
            b=VP+VW*(s[1]+1)-VW//4
            r=HP+HW*(s[2]+1)-HW//4

            if s[0]==0:   #No 1-cell shape -> nothing to draw
                pass
            elif s[0]==1: #Filled circle
                self.canvas.create_oval(t,l,b,r, fill=SHAPECOLOR, outline=SHAPECOLOR, width=SHAPEWIDTH)                
            elif s[0]==2: #Empty circle
                self.canvas.create_oval(t,l,b,r, outline=SHAPECOLOR, width=SHAPEWIDTH)
            elif s[0]==3: #Filled square
                self.canvas.create_rectangle(t,l,b,r, fill=SHAPECOLOR, outline=SHAPECOLOR, width=SHAPEWIDTH)                
            elif s[0]==4: #Empty square
                self.canvas.create_rectangle(t,l,b,r, outline=SHAPECOLOR, width=SHAPEWIDTH)
            elif s[0]==5: #Min cell
                self.canvas.create_line(t0+VW//2, l0+LW1, t0+VW//2, l0+SARROW+LW1, fill=SHAPECOLOR, width=LW2, arrow='first')
                self.canvas.create_line(t0+VW//2, l0+HW-LW1, t0+VW//2, l0+HW-SARROW-LW1, fill=SHAPECOLOR, width=LW2, arrow='first')
                self.canvas.create_line(t0+LW1, l0+HW//2, t0+SARROW+LW1, l0+HW//2, fill=SHAPECOLOR, width=LW2, arrow='first')
                self.canvas.create_line(t0+VW-LW1, l0+HW//2, t0+VW-SARROW-LW1, l0+HW//2, fill=SHAPECOLOR, width=LW2, arrow='first')
            elif s[0]==6: #Max cell        
                self.canvas.create_line(t0+VW//2, l0+LW1, t0+VW//2, l0+SARROW+LW1, fill=SHAPECOLOR, width=LW2, arrow='last')
                self.canvas.create_line(t0+VW//2, l0+HW-LW1, t0+VW//2, l0+HW-SARROW-LW1, fill=SHAPECOLOR, width=LW2, arrow='last')
                self.canvas.create_line(t0+LW1, l0+HW//2, t0+SARROW+LW1, l0+HW//2, fill=SHAPECOLOR, width=LW2, arrow='last')
                self.canvas.create_line(t0+VW-LW1, l0+HW//2, t0+VW-SARROW-LW1, l0+HW//2, fill=SHAPECOLOR, width=LW2, arrow='last')
            elif s[0]==7: #Encricled cell for summing arrow
                self.canvas.create_polygon([t0+2*LW2,l0+2*LW2,
                                       t0+VW-2*LW2,l0+2*LW2,
                                       t0+VW-2*LW2,l0+HW-2*LW2,
                                       t0+2*LW2,l0+HW-2*LW2], 
                                       outline=SHAPECOLOR, fill=BGCOLOR, width=LW1, smooth=True)
                
    def DrawSelection(self, selected):
        for (i,j) in selected:
            t=VP+VW*i
            l=HP+HW*j
            b=VP+VW*(i+1)
            r=HP+HW*(j+1)
            self.canvas.create_rectangle(t, l, b, r, fill=SELCOLOR, outline='', stipple=SELSTIPPLE)

    def DrawState(self,grid, allcpm):
        for i in range(9):
            for j in range(9):
                digit = grid.get_digit(i,j)
                cpm = grid.get_centerpm(i,j)
                allcpmtxt=''
                if cpm=='' and allcpm:
                    allcpmtxt = grid.get_centerpm(i,j, forced=True)
                tpm = grid.get_toppm(i,j)
                if digit!=0:       
                    color = FONTCOLOR
                    if grid.is_orig(i,j):
                        color = ORIGFONTCOLOR
                    self.canvas.create_text(VP+VW*i+VW/2,HP+HW*j+HW/2,
                                       anchor = "center",
                                       fill=color,
                                       font=FONT+" "+FONTSIZE,
                                       text=str(digit))
                else:
                    if cpm!='':                
                        self.canvas.create_text(VP+VW*i+VW/2-PM_ADJ,HP+HW*j+HW/2,
                                           anchor = "center",
                                           fill=FONTCOLOR,
                                           font=FONT_PM+" "+FONTSIZE_PM,
                                           text=cpm)
                    elif allcpmtxt!='':
                        self.canvas.create_text(VP+VW*i+VW/2-PM_ADJ,HP+HW*j+HW/2,
                                           anchor = "center",
                                           fill=INACTIVECPMCOLOR,
                                           font=FONT_PM+" "+FONTSIZE_PM,
                                           text=allcpmtxt)                                                
                    if tpm!='':
                        self.canvas.create_text(VP+VW*i+VW/2-PM_ADJ,HP+HW*j+HW/2-TOPPM_PADDING,
                                           anchor = "center",
                                           fill=FONTCOLOR,
                                           font=FONT_PM+" "+FONTSIZE_PM+ " italic",
                                           text=tpm)
                    if grid.hints[i,j]!='':
                        self.canvas.create_text(VP+VW*i+PM_ADJ,HP+HW*(j+1)-BOTTOMPM_PADDING+PM_ADJ,
                                           anchor = "nw",
                                           fill=HINTCOLOR,
                                           font=FONT_HINT+" "+FONTSIZE_HINT,
                                           text=grid.hints[i,j])
                        
                        
    def DrawFrame(self, i, j):
        if self.framelines:
            for e in self.framelines:
                self.canvas.delete(e)
                
        e = self.canvas.create_rectangle( VP+VW*i, HP+HW*j, 
                                     VP+VW*(i+1), HP+HW*(j+1), 
                                     outline=FRAMECOLOR, width=FRAMEWIDTH)
        self.framelines.append(e)        
        
    def Draw(self, gridlist, curri, currj, allcpm, currcolor, selected, delete=True):
        grid = gridlist.getcurr()
        if delete:
            self.canvas.delete('all')
        self.DrawColors(grid)
        self.DrawSelection(selected)        
        self.DrawBase(gridlist, currcolor)
        self.DrawShapes(grid)
        self.DrawState(grid,allcpm)
        self.DrawFrame(curri, currj)
    

In [7]:

class App:
    def __init__(self):
        self.allcpm = 0 #If showing all CenterPM
        
        self.currcolor = 0 #Index of current color (from COLORLIST)
        
        self.curri = 0
        self.currj = 0
        self.selected = set()            
    
        self.gridlist = GridList()       

        self.root = Tk()        

        self.canvas = Canvas(self.root, width=2*VP+9*VW+1, height=2*HP+9*HW+1, background=BGCOLOR)
        self.ui = UI(self.canvas)
        self.Draw(delete=False)
        self.ui.canvas_pack()
                
        #Key bindings
        #Arrow keys
        self.canvas.bind("<Up>",    self.key_arrow)
        self.canvas.bind("<Down>",  self.key_arrow)
        self.canvas.bind("<Left>",  self.key_arrow)
        self.canvas.bind("<Right>", self.key_arrow)
        
        #Specific keys & key combinations
        self.canvas.bind("u",           self.undo)
        self.canvas.bind("r",           self.redo)
        self.canvas.bind("A",           self.select_all)
        self.canvas.bind("<Control-a>", self.deselect_all)
        self.canvas.bind("o",           self.centerpm_onoff)
        self.canvas.bind("s",           self.show_all_centerpm)
        self.canvas.bind("w",           self.inc_showlevel)
        self.canvas.bind("q",           self.dec_showlevel)
        self.canvas.bind("c",           self.set_color)
        self.canvas.bind("C",           self.inc_currcolor)
        self.canvas.bind("f",           self.count_solution)
        self.canvas.bind("F",           self.find_solution)
        self.canvas.bind("<Control-c>", self.dec_currcolor)
        self.canvas.bind("<Alt-u>",     self.undo_base)
        self.canvas.bind("<Alt-r>",     self.redo_base)
        self.canvas.bind("<Alt-s>",     self.next_shape_base)
        self.canvas.bind("<Alt-l>",     self.next_line_segment_base)
        self.canvas.bind("<Alt-t>",     self.build_thermo_base)
        
        #All other keys (including digits)
        self.canvas.bind("<Key>",   self.key)
        
        #Mouse bindings
        self.canvas.bind("<Button-1>",         self.leftclick)
        self.canvas.bind("<Shift-Button-1>",   self.leftclick)
        self.canvas.bind("<Control-Button-1>", self.leftclick)
                        
        self.canvas.focus_set()
        
    def mainloop(self):
        self.root.mainloop()        
       
    def Draw(self, delete=True):
        self.ui.Draw(self.gridlist, self.curri, self.currj, self.allcpm, 
                self.currcolor, self.selected, delete)

        
    ##########################
    # Event handler methods    
    
    def key_arrow(self, event):
        oldi = self.curri
        oldj = self.currj
        if event.keycode==38:    #up
            self.currj-=1+9
            self.currj%=9
        elif event.keycode==40:  #down
            self.currj+=1
            self.currj%=9
        elif event.keycode==37:  #left
            self.curri-=1+9
            self.curri%=9
        elif event.keycode==39:  #right
            self.curri+=1
            self.curri%=9
        if event.state & 0x0001: #shift
            if not (oldi,oldj) in self.selected:
                self.selected.add( (oldi,oldj) )
            if not (self.curri,self.currj) in self.selected:
                self.selected.add( (self.curri,self.currj) )
            self.Draw()
        elif event.state & 0x0004: #control
            if (oldi,oldj) in self.selected:
                self.selected.remove( (oldi,oldj) )
            if (self.curri,self.currj) in self.selected:
                self.selected.remove( (self.curri,self.currj) )
            self.Draw()
        else:
            self.ui.DrawFrame(self.curri, self.currj)


    def key(self, event):
        # Numbers from 1 to 9
        if event.keycode>=49 and event.keycode<=57:            
            digit = event.keycode - 48
            if event.state & 0x0004: #control
                self.gridlist.change_toppm(self.selected, self.curri, self.currj, digit)
            elif event.state & 0x0001: #shift
                self.gridlist.delete_centerpm(self.selected, self.curri, self.currj, digit)
            elif event.state & 0x20000: #alt
                self.gridlist.add_base_digit(self.selected, self.curri, self.currj, digit)
            else: #no modifier
                self.gridlist.add_digit(self.selected, self.curri, self.currj, digit)            
        # Selecting current cell
        elif event.char==' ':
            if (self.curri,self.currj) in self.selected:
                self.selected.remove( (self.curri,self.currj) )
            else:
                self.selected.add( (self.curri,self.currj) )
        self.Draw()
        
    def undo(self, event):
        self.gridlist.undo()
        self.Draw()
        
    def redo(self, event):
        self.gridlist.redo()
        self.Draw()

    def undo_base(self, event):
        self.gridlist.undo_base()
        self.Draw()
        
    def redo_base(self, event):
        self.gridlist.redo_base()
        self.Draw()
        
        
    def select_all(self, event):
        self.selected = set()
        for i in range(9):
            for j in range(9):
                self.selected.add((i,j))
        self.Draw()

    def deselect_all(self, event):
        self.selected = set()
        self.Draw()
        
    def centerpm_onoff(self, event):
        self.gridlist.centerpm_onoff(self.selected, self.curri, self.currj)
        self.Draw()
    
    def show_all_centerpm(self, event):
        self.allcpm = 1 - self.allcpm
        self.Draw()
        
    def leftclick(self, event):
        x = event.x-HP
        y = event.y-VP
        i = x//HW
        j = y//VW
        if i>=0 and i<9 and j>=0 and j<9:
            self.curri = i
            self.currj = j
            if event.state & 0x0004: #control
                if (self.curri,self.currj) in self.selected:
                    self.selected.remove( (self.curri,self.currj) )
                self.Draw()                                            
            elif event.state & 0x0001: #shift
                if not (self.curri,self.currj) in self.selected:
                    self.selected.add( (self.curri,self.currj) )            
                self.Draw()                            
            else: # neither control nor shift
                self.ui.DrawFrame(self.curri, self.currj)            
    
    def inc_showlevel(self, event):
        self.gridlist.change_showlevel(1)
        self.Draw()
    
    def dec_showlevel(self, event):
        self.gridlist.change_showlevel(-1)
        self.Draw()
        
    def set_color(self, color):
        self.gridlist.change_color(self.selected, self.curri, self.currj, self.currcolor)
        self.Draw()
    
    def inc_currcolor(self, event):
        self.currcolor = (self.currcolor + 1) % 10 #Colors go from 0 to 9
        self.Draw()

    def dec_currcolor(self, event):
        self.currcolor = (self.currcolor - 1) % 10 #Colors go from 0 to 9
        self.Draw()
        
    def next_shape_base(self, event):
        if not self.selected:
            self.gridlist.next_1cellshape_base(self.curri, self.currj)
        elif len(self.selected)==2:
            l = list(self.selected)
            l.sort()
            i1=l[0][0]
            j1=l[0][1]
            i2=l[1][0]
            j2=l[1][1]
            self.gridlist.next_2cellshape_base(i1,j1,i2,j2)
        elif len(self.selected)==4:
            pass
        self.Draw()
        
    def next_line_segment_base(self, event):
        if len(self.selected)==2:
            l = list(self.selected)
            l.sort()
            i1=l[0][0]
            j1=l[0][1]
            i2=l[1][0]
            j2=l[1][1]
            self.gridlist.next_line_segment_base(i1,j1,i2,j2)
            self.Draw()
            
    def find_solution(self, event):
        nr,exact = self.gridlist.find_solution()
        self.Draw()
        self.show_sol_info(nr,exact)

    def count_solution(self, event):
        nr,exact = self.gridlist.count_solution()
        self.Draw()
        self.show_sol_info(nr,exact)
        
    def show_sol_info(self,nr,exact):
        if exact and nr<MAXSOLNR:
            showinfo(title='Solution', message='Number of solutions: {}.'.format(nr))        
        elif nr>=MAXSOLNR:
            showinfo(title='Solution', 
                     message='Number of solutions: {} or more.\nMaximum solution number has been reached.' \
                             .format(nr))        
        else:
            showinfo(title='Solution', 
                     message='Number of solutions: {} or more.\nMaximum number of node ({}) has been reached.' \
                             .format(nr, MAXNODENR))        
        
    def build_thermo_base(self, event):
        self.gridlist.build_thermo_base()
        self.Draw()
        
        


In [8]:
app = App()
app.mainloop()